# Automated testing of different hyperparameters

In [1]:
import classifier

import numpy as np
import matplotlib.pyplot as plt

2024-04-07 13:21:44.426615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
INPUT_WIDTH = 3

## Load data

In [3]:
players_inputs = classifier.players_inputs()
# don't use the test set here, that's cheating
(train_x, train_y), (valid_x, valid_y), (_, _) = classifier.prepare_data(players_inputs)

num_players = len(players_inputs.keys())

## Create model & run tests for one set of hyperparams

In [8]:
def test_params(seq_len, kern1, kern2, kern3, stride1, stride2, drop):
    model = classifier.createClassifier(INPUT_WIDTH, seq_len, kern1, kern2, kern3, stride1, stride2, drop)
    
    # train
    history = model.fit(
        train_x, train_y, epochs=60, verbose=0, batch_size=256, validation_data=(valid_x, valid_y)
    )

    # test
    valid_h = model.predict(valid_x)
    ranks = []

    for i in range(valid_h.shape[0]):
        rankings = np.argsort(valid_h[i])
        rank = (num_players-1) - np.where((rankings == np.argmax(valid_y[i])))[0][0]
        ranks.append(rank)

    topn_occurences = []
    running = 0
    for i in range(num_players):
        topn_occurences.append(ranks.count(i) + running)
        running += ranks.count(i)

    topn_acc = [t / topn_occurences[-1] for t in topn_occurences]
    
    return history, topn_acc

## Iterate over all our parameters

In [5]:
# k1_totest = [5, 7, 11]
k1_totest = [7, 11] # resuming after 5 is already done
k2_totest = [3, 5]
k3_totest = [2, 3, 4]
s1_totest = [2, 3]
s2_totest = [1, 2, 3]
d_totest = [0.3]

In [9]:
# oh god
for k1 in k1_totest:
    for k2 in k2_totest:
        for k3 in k3_totest:
            for s1 in s1_totest:
                for s2 in s2_totest:
                        print(f'testing {k1},{k2},{k3},{s1},{s2}...')
                        history, topn_acc = test_params(120, k1, k2, k3, s1, s2, 0.2)
                        
                        # save a training/validation accuracy plot
                        plt.figure()
                        plt.plot('val_accuracy', data=history.history)
                        plt.plot('accuracy', data=history.history)
                        plt.ylabel('accuracy')
                        plt.xlabel('epoch')

                        plt.title(f'kerns=({k1},{k2},{k3}), strides=({s1},{s2}), drop={0.2}')
                        plt.legend()
                        
                        int_d = int(d*10)
                        plt.savefig(f'search_plots/k{k1}{k2}{k3}_s{s1}{s2}_d{int_d}.png')

                        plt.close()

                        # write to the output csv
                        with open('results.csv', 'a') as res_file:
                            new_row = f'\n{k1},{k2},{k3},{s1},{s2},{0.2},{topn_acc[0]},{topn_acc[1]},{topn_acc[2]}'
                            res_file.write(new_row)


testing 7,3,2,2,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step
testing 7,3,2,2,2...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
testing 7,3,2,2,3...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
testing 7,3,2,3,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
testing 7,3,2,3,2...
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
testing 7,3,2,3,3...
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
testing 7,3,3,2,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
testing 7,3,3,2,2...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
testing 7,3,3,2,3...
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
testing 7,3,3,3,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
testing 7,3,3,3,2...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
testing 7,3,3,3,3...
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
testing 7,3,4,2,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
testing 7,3,4,2,2...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
testing 7,3,4,2,3...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
testing 7,3,4,3,1...
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
testing 7,3,4,3,2...
95/